In [ ]:
# process_nutrient_example
# ---
# NaaVRE:
#  cell:
#   inputs: 
#    - P_loads: List
#   outputs: 
#    - new_Ploads: List
#   params: []
#   secrets: []
#   dependencies: []
# ...

# P_loads = list(1,2,3)
# P_loads

new_Ploads = list()
for (P_load in P_loads){
    new_Pload = P_load*2000+1
    new_Ploads = append(new_Ploads, new_Pload)
}
    

In [ ]:
# print_nutrient_example
# ---
# NaaVRE:
#  cell:
#   inputs: 
#    - new_Ploads: List
#   outputs: []
#   params: []
#   secrets: []
#   dependencies: []
# ...

print(new_Ploads)

In [ ]:
# Cell_2_PCLake_bifurcation
# ---
# NaaVRE:
#  cell:
#   inputs:
#    - Bifur_PLoads: List
#   outputs: 
#    - bifur_output: List
#   params: []
#   secrets: []
#   dependencies: []
# ...

                         
                                    
dest_dir  = "/tmp/data/PCLake_NaaVRE"   

PCLake_naavre_function = function(PLoad, dest_dir){
 local({
   
   #    - result_filename: String
   
   # PCLake single run of all state and parameter sets			
   # author: Sven Teurlincx (June 2017), based on Luuk van Gerven	(November 2015)
   # used libraries: deSolve (to run PCLake)
   
   # user defined settings
   # dest_dir  = "/tmp/data/PCLake_NaaVRE" 
   # dir_SCHIL           =	"D:/Users/SvenT/Documents/PCLake/PCLake_3162/PCModel1350/PCModel/3.00/Models/PCLake/6.13.16/PCShell/"	# location of PCShell
   dir_SCHIL =	paste0(dest_dir,"/PCModel/Licence_agreement/I_accept/PCModel1350/PCModel/3.00/Models/PCLake/6.13.16/PCShell/")	# location of PCShell
   
   #dir_DATM			=	"D:/Users/SvenT/Documents/PCLake/PCLake_3162/PCModel1350/PCModel/3.00/"					# location of DATM implementation (excel)
   dir_DATM = paste0(dest_dir,"/PCModel/Licence_agreement/I_accept/PCModel1350/PCModel/3.00/")					# location of DATM implementation (excel)
   
   #file_DATM =	"PL613162.xls"																			# file name of the DATM implementation
   file_DATM	=	paste0(dest_dir,"/PCModel/Licence_agreement/I_accept/PCModel1350/PCModel/3.00/Models/PCLake/6.13.16/PL613162.xls") # file name of the DATM implementation
   work_case   =	"R_base_work_case"                      												# name of work case
   modelname 	=	"_org"																					# name of the model (suffix to specific model files)
   
   #define the number of logical processers (cores) to use for the computation
   nCORES	=	4
   
   #Set to true if the initialisation should output all states, parameters and auxiliaries
   tGENERATE_INIT	=	FALSE
   
   # # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------
   # # initialisation of PCLake R Shell
   # # -----------------------------------------------------------------------------------------------------------------------------------------------------------------------
   source(paste(dir_SCHIL,"scripts/R_system/functions.r",sep=""), local = TRUE)  					 # Define functions
   # #here we load the cpp files containing the model code (the equations and initial settings) 
   # #	these are loaded in the script as the user may wish to create multiple different models (different cpp's) and compare them
   # #	In that case the user will have to compile multiple different DATM instances and save the cpp files to different folders,
   # #		the names of which can be looped through 
   cpp_files <- list.files(file.path(dir_DATM,paste("Frameworks/Osiris/3.01/PCLake/",sep="")), full.names = TRUE)[
     which((lapply(strsplit(x=list.files(file.path(dir_DATM,paste("Frameworks/Osiris/3.01/PCLake/",sep="")), full.names = TRUE), split="[/]"), 
                   function(x) which(x %in% c("pl61316ra.cpp","pl61316rc.cpp","pl61316rd.cpp","pl61316ri.cpp","pl61316rp.cpp","pl61316rs.cpp",
                                              "pl61316sa.cpp","pl61316sc.cpp","pl61316sd.cpp","pl61316si.cpp","pl61316sp.cpp","pl61316ss.cpp")))>0)==TRUE)]		
   file.copy(cpp_files, file.path(dir_SCHIL, work_case,"source_cpp"),overwrite=T)
   
   source(paste(dir_SCHIL,"scripts/R_system/201703_initialisationDATM.r",sep=""), local = TRUE)    	 # Initialisation (read user defined input + convert cpp files of model + compile model)
   
   
   # ***********************************************************************
   # -----------------------------------------------------------------------
   # Initialize Model (calculate state variables at t=0; stored in 'states')
   # -----------------------------------------------------------------------
   # ***********************************************************************
   
   WriteLogFile(LogFile,ln="- initialize model")
   dfSTATES_INIT_T0	= 	as.data.frame(dfSTATES[,which(colnames(dfSTATES) %in% c('iReportState','sInitialStateName'))])
   dfSTATES_INIT		=	as.data.frame(dfSTATES[,-which(colnames(dfSTATES) %in% c('iReportState','sInitialStateName'))])
   for (nSET in 1:ncol(dfSTATES_INIT)){
     
     vSTATES_LIST		=	dfSTATES_INIT[,nSET]
     names(vSTATES_LIST)	=	dfSTATES$sInitialStateName
     InitializeModel(n_states, vSTATES_LIST)
     dfSTATES_INIT_T0=cbind.data.frame(dfSTATES_INIT_T0,states)
   }
   colnames(dfSTATES_INIT_T0)=colnames(dfSTATES)
   
   dfPARAMS_INIT	=	as.data.frame(dfPARAMS[,-which(colnames(dfPARAMS) %in% c('iReport','sMinValue','sMaxValue')),drop=F])
   
   nSET = 1                             
   # Provide the model with the parameter settings of the given set
   new_pars     =	dfPARAMS_INIT[,nSET]
   names(new_pars) <- rownames(dfPARAMS_INIT) 
   
   new_pars["ReadPLoad"] = 1
   new_pars["mPLoad"] = PLoad
   
   # Provide the model with the initial states of the given set  
   new_states	=	dfSTATES_INIT_T0[,nSET+2]
   names(new_states) <- rownames(dfSTATES_INIT_T0) 
   
   # run PCLake and store the output
   int        <- "vode"
   error      <- class(tryCatch(output <- as.data.frame(RunModel(new_states,times,new_pars,forcings,aux_number,aux_names,"vode",state_names,internal_time_step)),error = function(e) e))[1] == "simpleError"
   output	   <- as.data.frame(subset(output, , subset=(time %in% c((fREP_START_YEAR*365):max(times)))))
   if(any(is.na(output)) | error) {  # run the model again when integrator "vode" returns negative or NA outputs, rerun with integrator "daspk"
     int        <- "daspk"
     error      <- class(tryCatch(output <- as.data.frame(RunModel(new_states,times,new_pars,forcings,aux_number,aux_names,"daspk",state_names,internal_time_step)),error = function(e) e))[1] == "simpleError"
     output	   <- as.data.frame(subset(output, subset=(time %in% c((fREP_START_YEAR*365):max(times)))))
     if(any(is.na(output)) | error) { # run the model again when integrator "daspk" returns negative or NA outputs, rerun with integrator "euler"
       int        <- "euler"
       error      <- class(tryCatch(output <- as.data.frame(RunModel(new_states,times,new_pars,forcings,aux_number,aux_names,"euler",state_names,0.003)),error = function(e) e))[1] == "simpleError"
       output	   <- as.data.frame(subset(output, subset=(time %in% c((fREP_START_YEAR*365):max(times)))))
       if(any(is.na(output)) | error) { # run the model again when integrator "euler" returns negative or NA outputs, rerun with integrator "euler" with timesept 0.002
         error      <- class(tryCatch(output <- as.data.frame(RunModel(new_states,times,new_pars,forcings,aux_number,aux_names,"euler",state_names,0.002)),error = function(e) e))[1] == "simpleError"
         output	   <- as.data.frame(subset(output, subset=(time %in% c((fREP_START_YEAR*365):max(times)))))
       }
     }
   }
   
   #	#single run
   #	dyn.load("model.dll")
   #	single_run=ode(y=new_states, times=times, func = "CalculateDerivatives", parms = new_pars, dllname = "model",initforc = "forcc",forcings=forcings,initfunc = "initmod", nout = aux_number, outnames = aux_names, method = int)
   #	dyn.unload("model.dll")
   
   #write output to dataframe
   dfOUTPUT_FINAL	=	cbind.data.frame(PLoad = PLoad, nParamSet=nSET, nStateSet=nSET, output)
   
   variables <- ls(envir = .GlobalEnv)
   
   rm(list = variables[-which(variables=="dfOUTPUT_FINAL")])
   return(dfOUTPUT_FINAL) 
 })
}
                             

#########################################
####### bifurcation analysis ############
#########################################
bifur_output = list()
# Bifur_PLoads <- list(0.0001,0.001, 0.002, 0.003, 0.005) # P loading in gP/m2/day
for (n in 1:length(Bifur_PLoads)){
 PLoad = Bifur_PLoads[[n]]
#  for (nSET in 1:ncol(dfPARAMS_INIT)) {      # loop over sets
    
    dfOUTPUT_FINAL = PCLake_naavre_function(PLoad=PLoad, dest_dir=dest_dir)
    
    # WriteLogFile(LogFile,ln=paste("Initials recorded for Set_",nSET-1,sep=""))
  #}
    # write.table(x=dfOUTPUT_FINAL, file=paste(dir_SCEN,"results/","singlerun_",work_case,".csv",sep=""),sep=',',row.names=FALSE, col.names = TRUE, quote = FALSE) 	
    output_folder= paste0("/tmp/data/bifurcation_output/Pvalue_",n)
    # paste("/tmp/data/bifurcation_output/Pvalue_",n,".csv",sep="")
    # Ensure the local folder exists
    if (!dir.exists(output_folder)) {
      dir.create(output_folder, recursive = TRUE)
    }
    output_filename = paste0(output_folder,"/PLoad_",PLoad,".csv")                         
	write.csv(x=dfOUTPUT_FINAL, file= output_filename,sep=',',row.names=FALSE, col.names = TRUE, quote = FALSE) 
    # bifur_output = append(bifur_output, 
	# paste(dir_SCEN,"results/","singlerun_",work_case,".csv",sep=""))
    head(dfOUTPUT_FINAL)
    bifur_output = append(bifur_output, output_filename)
    
    variables <- ls(envir = .GlobalEnv)
    print(variables)
    # rm(list = variables[-which(variables%in%c("dest_dir","Bifur_PLoads","bifur_output","n","dfOUTPUT_FINAL","PCLake_naavre_function"))])
}


bifur_output



In [ ]:
# cell3_print_bifurcation_results
# ---
# NaaVRE:
#  cell:
#   inputs: 
#    - bifur_output: List
#   outputs: 
#    - combined_df_filename: String
# ...


# plot the bifurcation analysis
# 1 merge output data frames

combined_df <- data.frame()
for (df_output in bifur_output){
    df <- read.csv(df_output)
    combined_df <- rbind(combined_df, df)

}
# Print the combined data frame
print(head(combined_df))

# combined_df_filename = "/home/jovyan/PCLake_Naavre/PCLake_output.csv" 
combined_df_filename = "/tmp/data/PCLake_output.csv" 
write.csv(combined_df, file= combined_df_filename, row.names=FALSE, col.names = TRUE, quote = FALSE)

In [ ]:
# cell4_print_bifurcation_results
# ---
# NaaVRE:
#  cell:
#   inputs: 
#    - combined_df_filename: String
#   outputs: []
#   params:
#    - param_s3_server: String
#    - param_s3_user_prefix: String
#   secrets:
#    - secret_s3_access_key: String
#    - secret_s3_secret_key: String
# ...

# 1 create plot     
comb_df = read.csv(combined_df_filename)
PLoad_vec = unique(comb_df$PLoad)

figname_PCLake_PLoads = "/tmp/data/PCLake_PLoads.png" 
# figname_PCLake_PLoads = "/home/jovyan/PCLake_Naavre/PCLake_PLoads.png" # save your plot in the current folder within virtual lab

png(figname_PCLake_PLoads, width=900, height=600)
library(ggplot2)

# Create a data frame for plotting

# Plot using ggplot2
comb_df$PLoad = round(comb_df$PLoad, digits=4)
ggplot(comb_df, aes(x = time, y = oChla, color = factor(PLoad))) +
  geom_line() +
  geom_point(size = 0.2) +
  scale_color_manual(values = 1:length(PLoad_vec), 
                     labels = paste0("PLoad=", PLoad_vec, " gP/m2/day")) +
  labs(y = "oChla [mg/m3]", x = "time", color = "PLoad")  +
  theme_minimal() +
  theme(legend.position = "right", aspect.ratio=1/2, plot.margin=unit(c(0.1, 0.1, 0.1, 0.1),"cm"))
dev.off()

# 2 upload file
devtools::install_github("cboettig/minioclient")

library(minioclient)
install_mc()

# This is needed to for minioclient to work once the cell containerized.
# It will fail when running in a notebook, but we ignore the error
library(processx)
processx::run("apt-get", "update", error_on_status=FALSE)
processx::run("apt-get", c("install", "-y", "ca-certificates"), error_on_status=FALSE)

mc_alias_set(
    alias = "scruffy",
    endpoint = param_s3_server,
    access_key = secret_s3_access_key,
    secret_key = secret_s3_secret_key,
)
  
mc_cp(combined_df_filename, paste0("scruffy/naa-vre-user-data/", param_s3_user_prefix, "/PCLake_output.csv"))     
mc_cp(figname_PCLake_PLoads, paste0("scruffy/naa-vre-user-data/", param_s3_user_prefix, "/PCLake_PLoads.png"))
